In [7]:
#= 📦 Importy i załadowanie danych =#
using JLD2
using Printf
using Statistics
using LinearAlgebra

include("src/AD.jl")
include("src/NN.jl")

Main.NN

In [ ]:
#= 🔄 Przygotowanie danych =#
file = load("data/imdb_dataset_prepared.jld2")
X_train = Int.(file["X_train"])
y_train = vec(Float32.(file["y_train"]))
X_test  = Int.(file["X_test"])
y_test  = vec(Float32.(file["y_test"]))
embeddings = file["embeddings"]
vocab = file["vocab"]
embedding_dim = size(embeddings, 1)

50

In [9]:
dataset = NN.DataLoader((X_train, y_train), batchsize=64, shuffle=true)

Main.NN.DataLoader((Float32[6391.0 286.0 … 11382.0 8311.0; 143.0 286.0 … 4254.0 2891.0; … ; 12849.0 12849.0 … 12849.0 12849.0; 12849.0 12849.0 … 12849.0 12849.0], Float32[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]), 64, true, [32683, 39034, 7812, 9682, 38165, 8530, 24884, 13800, 15857, 24336  …  38836, 21648, 23095, 25678, 26108, 32282, 33919, 10887, 16630, 39123])

In [ ]:
model = NN.Chain(
    x -> Int.(x),
    NN.Embedding(length(vocab), embedding_dim, embeddings),
    x -> permutedims(x, (2, 1, 3)),
    NN.Conv1D(3, embedding_dim, 8, NN.relu),
    NN.MaxPool1D(8),
    NN.flatten,
    NN.Dense(128, 1, NN.sigmoid)
)

Main.NN.Chain(Any[var"#5#7"(), Main.NN.Embedding(12849, 50, Float32[0.90951 -0.58014 … 0.014323 0.0; -0.20702 -1.1316 … -0.74624 0.0; … ; -0.60396 0.1722 … -0.079873 0.0; -0.31136 1.2622 … -0.89164 0.0], Int64[]), var"#6#8"(), Main.NN.Conv1D(3, 50, 8, Float32[0.19548656 -0.13748096 … -0.029870939 -0.13235855; -0.047251515 -0.034629982 … 0.01699292 0.1243569; 0.023121303 0.0060622953 … -0.033419542 -0.10284888;;; 0.14944689 -0.10561571 … -0.08841027 -0.16341622; -0.17465019 0.0412365 … -0.026658136 -0.122651815; -0.09217791 -0.12946326 … -0.11048508 0.094671965;;; 0.054223638 -0.11854025 … 0.011167886 -0.0025306377; 0.022569947 -0.058938477 … -0.34426907 -0.08609958; 0.0028482324 0.02538908 … 0.11023644 0.24970715;;; -0.060148705 0.084308445 … -0.0178336 -0.11965072; 0.02093437 0.17145135 … 0.11038865 -0.08106754; 0.120033376 -0.0042178603 … -0.09970752 -0.08653182;;; -0.15025014 0.113206446 … -0.06264779 0.029800653; -0.2311311 -0.030981781 … -0.07136277 -0.06504426; 0.03893273 0.11978

In [11]:
accuracy(m, x, y) = mean((vec(m(x)) .> 0.5) .== (y .> 0.5))

accuracy (generic function with 1 method)

In [12]:
#= 🚀 Trening modelu =#
opt = NN.Adam(Float32(0.001))
epochs = 5
NN.train_model(model, dataset, X_test, y_test, opt, epochs)

Epoch: 1 (20.18s) 	Train: (l: 0.59, a: 0.69) 	Test: (l: 0.50, a: 0.76)
Epoch: 2 (20.57s) 	Train: (l: 0.48, a: 0.77) 	Test: (l: 0.46, a: 0.79)
Epoch: 3 (19.73s) 	Train: (l: 0.46, a: 0.79) 	Test: (l: 0.44, a: 0.80)
Epoch: 4 (21.50s) 	Train: (l: 0.44, a: 0.80) 	Test: (l: 0.43, a: 0.80)
Epoch: 5 (20.03s) 	Train: (l: 0.43, a: 0.81) 	Test: (l: 0.43, a: 0.81)
